In [1]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from typing import TypedDict, List, Union

from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

from dotenv import load_dotenv

load_dotenv()

class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]

llm = ChatOpenAI(model="gpt-4o")

def process(state: AgentState) -> AgentState:
    """This node will solve the request you input"""
    response = llm.invoke(state["messages"])
    
    state["messages"].append(AIMessage(content=response.content))
    print(f"\nAI: {response.content}")
    
    return state


graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()

conversation_history = []

user_input = input("Enter: ")
while user_input != "exit":
    conversation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({"messages": conversation_history})
    conversation_history = result["messages"]
    print(conversation_history)
    user_input = input("Enter: ")

Enter:  My name is Juan. I'd like to have a summary of the England History



AI: Certainly, Juan! Here's a brief summary of England's history:

1. **Prehistoric and Roman Britain**: Before the arrival of the Romans, Britain was home to various Celtic tribes. The Romans invaded in AD 43 and established the province of Britannia, leaving a legacy of roads, baths, and Hadrian's Wall to defend against northern tribes.

2. **Anglo-Saxon and Viking Age**: After the Roman withdrawal around AD 410, the Anglo-Saxons from present-day Germany, Denmark, and the Netherlands settled in England. This period saw the establishment of several kingdoms. Viking raids began in the late 8th century, leading to parts of England coming under Danish control.

3. **Norman Conquest and the Middle Ages**: In 1066, William the Conqueror from Normandy defeated Harold II at the Battle of Hastings, marking the start of Norman rule. This era saw the establishment of feudalism, the construction of castles, and the Domesday Book. The subsequent centuries were marked by conflicts such as the Hun

Enter:  Who was the first king?



AI: The first king to rule a united England was King Æthelstan. He reigned from 924 to 939 AD and is often considered the first true king of England as he successfully consolidated several smaller kingdoms under his rule. Æthelstan was the grandson of Alfred the Great, the famous King of Wessex. Before Æthelstan, what we now know as England was divided into several smaller Anglo-Saxon kingdoms, such as Wessex, Mercia, and Northumbria. His victory at the Battle of Brunanburh in 937 AD was a significant moment in his reign, as it helped to secure his authority over the entirety of England and fend off threats from other regional rulers and Viking invaders.
[HumanMessage(content="My name is Juan. I'd like to have a summary of the England History", additional_kwargs={}, response_metadata={}), AIMessage(content="Certainly, Juan! Here's a brief summary of England's history:\n\n1. **Prehistoric and Roman Britain**: Before the arrival of the Romans, Britain was home to various Celtic tribes. 

Enter:  exit


In [4]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

# Load environment variables
load_dotenv()

# Define the agent state with Annotated messages field
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# Define tools that the agent can use
@tool
def add(a: int, b: int):
    """This is an addition function that adds 2 numbers together"""
    return a + b

@tool
def subtract(a: int, b: int):
    """Subtraction function"""
    return a - b

@tool
def multiply(a: int, b: int):
    """Multiplication function"""
    return a * b

# Create tools list
tools = [add, subtract, multiply]

# Create the model with tools bound
model = ChatOpenAI(model="gpt-4o").bind_tools(tools)

# Define the main agent node
def model_call(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(content=
        "You are my AI assistant, please answer my query to the best of your ability."
    )
    response = model.invoke([system_prompt] + state["messages"])
    return {"messages": [response]}

# Define conditional logic for continuing or ending
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"

# Create the graph
graph = StateGraph(AgentState)
graph.add_node("our_agent", model_call)

# Create tool node for handling tool calls
tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

# Set entry point
graph.set_entry_point("our_agent")

# Add conditional edges
graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    },
)

# Add edge from tools back to agent
graph.add_edge("tools", "our_agent")

# Compile the graph
app = graph.compile()

# Utility function for printing stream output
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [5]:
# Example usage with simple math
print("=== Simple Math Example ===")
inputs = {"messages": [("user", "Add 3 + 4.")]}
print_stream(app.stream(inputs, stream_mode="values"))

=== Simple Math Example ===
================================ Human Message =================================

Add 3 + 4.
================================== Ai Message ==================================
Tool Calls:
  add (call_NNpAQXvZEG14HfAKHLYTz2W7)
 Call ID: call_NNpAQXvZEG14HfAKHLYTz2W7
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The result of adding 3 + 4 is 7.


In [6]:
print("\n=== Complex Math Example ===")
inputs = {"messages": [("user", "Add 34 + 21. Add 3 + 4. Add 12 + 12")]}
print_stream(app.stream(inputs, stream_mode="values"))


=== Complex Math Example ===
================================ Human Message =================================

Add 34 + 21. Add 3 + 4. Add 12 + 12
================================== Ai Message ==================================
Tool Calls:
  add (call_7KSkFLK8UtCmGKSsxbqvy4OU)
 Call ID: call_7KSkFLK8UtCmGKSsxbqvy4OU
  Args:
    a: 34
    b: 21
  add (call_dbIzZKp9KZ69QBtSnd5SpsYc)
 Call ID: call_dbIzZKp9KZ69QBtSnd5SpsYc
  Args:
    a: 3
    b: 4
  add (call_o1dP35wx7WKluGFUHpbnMQwy)
 Call ID: call_o1dP35wx7WKluGFUHpbnMQwy
  Args:
    a: 12
    b: 12
================================= Tool Message =================================
Name: add

24
================================== Ai Message ==================================

The results of the additions are as follows:
- 34 + 21 = 55
- 3 + 4 = 7
- 12 + 12 = 24
